Computing bids for the hybrid model

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime

In [7]:
df = pd.read_csv("Root_Insurance_data.csv")

Get probabilities via logistic regression using features 'Currently Insured_Y', number of vehicles, and number of drivers as categorical variables.

In [8]:
from sklearn.linear_model import LogisticRegression

glm = LogisticRegression()
glm.fit(
    pd.get_dummies(df[["Currently Insured","Number of Vehicles","Number of Drivers"]],
        columns=["Currently Insured","Number of Vehicles","Number of Drivers"])\
    .loc[df.click, ["Currently Insured_Y","Number of Vehicles_2","Number of Vehicles_3","Number of Drivers_2"]].to_numpy(),
    df["policies_sold"].loc[df.click].to_numpy()
)

LogisticRegression()

Get list of customer probabilities to buy and parameters for competing bid distributions.

In [9]:
# number of categories
cat = 36
total = np.zeros(cat) # total number in each category
bid = 10*np.ones(cat) # default bids set to 10
probSoldGivenClick = np.zeros(cat)
# parameters for the hybrid distribution
_lambda = np.zeros(cat)
_p = np.zeros(cat)


theta = df.groupby("rank").click.mean().to_numpy() # get estimates for click rates by rank

# add column to track indexnumber
df["class"] = 0
idx = 0
ls = pd.DataFrame( columns = ["insured", "cars", "drivers","married"])
for i in ["Y","N","unknown"]:
    for c in [1,2,3]:
        for d in [1,2]:
            for m in ["M","S"]:
                probSoldGivenClick[idx] = glm.predict_proba(
                    np.array([[(i=="Y"),(c==2),(c==3),(d==2)]])
                    )[0,1]
                # record index number
                df.loc[(df["Currently Insured"]==i)&(df["Number of Vehicles"]==c)&\
                           (df["Number of Drivers"]==d)&(df["Marital Status"]==m), "class"] = idx
                total[idx] = df.loc[ (df["Currently Insured"]==i)&(df["Number of Vehicles"]==c)&\
                           (df["Number of Drivers"]==d)&(df["Marital Status"]==m) ].shape[0]
                r = df["rank"].loc[ df["class"]==idx ].mean()
                if np.isnan(r):
                    r = df["rank"].mean()
                _p[idx] = (r-1)/4
                _lambda[idx] = 10*(r-1)/(5-r)
                ls = ls.append({"insured":i, "cars":c, "drivers":d, "married":m}, ignore_index=True)
                idx+=1
                

In [10]:
#expectedcost, expectedsold, expected cost per policy sold
def expectedsold(bid):#expected number of policies sold
    for i in range(0, len(bid)):
        if bid[i]<0:
            return 0
    e = 0
    for i in range(0, 35):#35 because click at 36 = 0, lambda will cause errors
        for r in range(1, 6):
            if bid[i]<0:
                e = e +0
            elif bid[i]<10: # when bid is less htan 10, basically just uniform
                e = e + total[i]*probSoldGivenClick[i]*theta[r-1]*math.comb(4, r-1)*\
                    (1 - bid[i]*(1-_p[i])/(10))**(r-1)*(bid[i]*(1-_p[i])/(10))**(5-r)
            else: #some aspects of uniform and exponential
                e = e + total[i]*probSoldGivenClick[i]*theta[r-1]*math.comb(4, r-1)*\
                    (_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(r-1)*(1-_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(5-r)
    return e
def expectedcost(bid):#expected cost of policy
    e = 0
    for i in range(0, 35):
        for r in range(1, 6):
            if bid[i]<0:
                e = e+0
            elif bid[i]<10: # when bid is less htan 10, basically just uniform
                e = e + total[i]*bid[i]*theta[r-1]*math.comb(4, r-1)*\
                    (1 - bid[i]*(1-_p[i])/(10))**(r-1)*(bid[i]*(1-_p[i])/(10))**(5-r)
            else: #some aspects of uniform and exponential
                e = e + total[i]*bid[i]*theta[r-1]*math.comb(4, r-1)*\
                    (_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(r-1)*(1-_p[i]*np.exp((-bid[i]+10)/_lambda[i]))**(5-r)
        #print(e)
    return e

def costpersold(bid):
    return expectedcost(bid)/expectedsold(bid)
def constraint(bid): #constaining that the expected number of policies sold is more than 400
    return expectedsold(bid)


In [11]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint, NonlinearConstraint

In [12]:
#hessian is zero matrix if needed
def cons_H(x, v):
    return np.zeros((36,36))
#nonlinar constraint is the constraint function is bounded from 400 to 1000
nonlinear_constraint = NonlinearConstraint(constraint, 400, 1000)#, hess=cons_H)
#linear constraint is each bid is between 1 to 50
lincon = LinearConstraint(np.identity(36), np.linspace(0.01, 0.01, num=36), np.linspace(50000, 50000, num=36))

In [13]:
# get constrained optimization 
nonlin =[]
for i in range(2):
    nonlin.append(NonlinearConstraint(constraint,400+i*100,400+i*100, hess=cons_H))
    
res = []
print(datetime.datetime.now())
for i in range(2):
    res.append(minimize(expectedcost, bid, method='trust-constr', constraints=[lincon, nonlin[i]],  options={'maxiter':5000})) 
    print(datetime.datetime.now())
for i in range(2):
    print(expectedcost(res[i].x), expectedsold(res[i].x), costpersold(res[i].x))

2021-05-28 20:18:56.339421
2021-05-28 20:19:43.841856
2021-05-28 20:20:24.799745
5176.368810228414 399.99999999999994 12.940922025571037
7924.4603780380385 499.99999999999994 15.848920756076078


In [14]:
ls["bid"] = res[0].x
ls.sort_values("bid",ascending=False)

,insured,cars,drivers,married,bid
35,unknown,3,2,S,9.947015
25,unknown,1,1,S,8.158637
24,unknown,1,1,M,8.112477
26,unknown,1,2,M,6.890346
27,unknown,1,2,S,6.875374
15,N,1,2,S,6.303553
13,N,1,1,S,6.292790
14,N,1,2,M,6.292443
29,unknown,2,1,S,6.201643
28,unknown,2,1,M,6.155251


Estimate the variance using bootstrapping from the original data set

In [15]:
niter = 10000
arr = np.zeros(niter)
for i in range(niter):
    batch = np.random.choice(df["class"], size=10000, replace=True)
    # need to modify totals 
    for j in range(cat):
        total[j] = np.count_nonzero(batch==j)
    arr[i] = expectedsold(res[0].x)
m = np.mean(arr)
s = np.std(arr)
print("Mean: ",m)
print("Standard Deviation: ",s)

Mean:  399.9894873228913
Standard Deviation:  3.794106119711101


In [16]:
from scipy.stats import norm

norm.cdf(400, loc=m,scale=s)

0.5011053844779731

Increaste the target expected sales to 406 to ensure the observed value will be at least 400

In [17]:
# get constrained optimization 
nonlin =[]
nonlin.append(NonlinearConstraint(constraint,406,406, hess=cons_H))
    
res = []
print(datetime.datetime.now())
res.append(minimize(expectedcost, bid, method='trust-constr', constraints=[lincon, nonlin[0]],  options={'maxiter':5000})) 
print(datetime.datetime.now())
print(expectedcost(res[0].x), expectedsold(res[0].x), costpersold(res[0].x))

2021-05-28 20:21:14.507493
2021-05-28 20:22:32.465799
5321.877833256757 405.99999999999994 13.108073480927976


In [18]:
ls["bid"] = res[0].x
ls.sort_values("bid",ascending=False)

,insured,cars,drivers,married,bid
35,unknown,3,2,S,9.942599
25,unknown,1,1,S,8.243087
24,unknown,1,1,M,8.196866
26,unknown,1,2,M,6.963841
27,unknown,1,2,S,6.948803
15,N,1,2,S,6.372905
13,N,1,1,S,6.362735
14,N,1,2,M,6.361692
29,unknown,2,1,S,6.268718
28,unknown,2,1,M,6.222026


In [22]:
niter = 10000
arr = np.zeros(niter)
for i in range(niter):
    batch = np.random.choice(df["class"], size=10000, replace=True)
    # need to modify totals 
    for j in range(cat):
        total[j] = np.count_nonzero(batch==j)
    arr[i] = expectedsold(res[0].x)
m = np.mean(arr)
s = np.std(arr)
print("Mean: ",m)
print("Standard Deviation: ",s)

Mean:  404.9035278694814
Standard Deviation:  3.8033885621576133


Now we achieve 90% confidence that the observed number will be greater than 400

In [23]:
from scipy.stats import norm

norm.cdf(400, loc=m,scale=s)

0.09865517496295945

In [21]:
import statsmodels.api as sm

lm = sm.OLS(ls["bid"],sm.add_constant(pd.get_dummies(ls[["insured","cars","drivers","married"]]))).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    bid   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.833
Method:                 Least Squares   F-statistic:                     30.16
Date:                Fri, 28 May 2021   Prob (F-statistic):           3.32e-11
Time:                        20:22:48   Log-Likelihood:                -48.177
No. Observations:                  36   AIC:                             110.4
Df Residuals:                      29   BIC:                             121.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.5631      0.064     